In [2]:
import keras
import numpy as np
from matplotlib import pyplot as plt
import cv2
from pathlib import Path

In [3]:
paths = ["data/train/normal", "data/train/covid19"]  
imgs = []  
labels = [] 

for idx, path in enumerate(paths):
    path_images = Path(path).glob("**/*.jpg")
    path_images = [str(x) for x in path_images]
    
    for path_image in path_images:
        # Leer la imagen en modo de escala de grises
        img = cv2.imread(path_image, 0)  
        # Normalizar la imagen
        img = img / 255  
        # Redimensionar y añadir a la lista
        imgs.append(cv2.resize(img, (150, 150), cv2.INTER_AREA))  
    
    # Añadir etiquetas correspondientes
    labels.extend([idx] * len(path_images))  

# Convertir las listas a arreglos de NumPy
imgs = np.array(imgs).reshape(-1, 150, 150, 1)
labels = np.array(labels)

print(f"Images shape: {imgs.shape}")
print(f"Num. of labels: {len(labels)}")

Images shape: (48, 150, 150, 1)
Num. of labels: 48


In [4]:
model = keras.Sequential([
    # Capa de entrada que recibe imágenes en escala de grises de tamaño 150x150
    keras.layers.Input((150, 150, 1)),  
    
    # Primera capa de pooling para reducir las dimensiones espaciales
    keras.layers.MaxPooling2D(2, 2),  

    # Capa convolucional con 32 filtros para extraer características
    keras.layers.Conv2D(32, (3, 3), padding="same", activation="relu"),  
    # Segunda capa de pooling para seguir reduciendo las dimensiones
    keras.layers.MaxPooling2D(2, 2),  

    # Capa Dropout para reducir el sobreajuste
    keras.layers.Dropout(0.5),  
    # Aplanar la salida para convertirla en un vector
    keras.layers.Flatten(),  
    # Capa totalmente conectada con 128 neuronas y activación ReLU
    keras.layers.Dense(128, activation="relu"),  

    # Capa de salida con 2 neuronas y activación softmax para clasificación binaria
    keras.layers.Dense(2, activation="softmax")
])

# Imprimir el resumen del modelo
print(model.summary())


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ max_pooling2d (MaxPooling2D)    │ (None, 75, 75, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 75, 75, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 37, 37, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 37, 37, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 43808)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     5,607,552 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,608,130 (21.39 MB)

 Trainable params: 5,608,130 (21.39 MB)

 Non-trainable params: 0 (0.00 B)

None


In [5]:
model.compile(loss=keras.losses.SparseCategoricalCrossentropy(), optimizer=keras.optimizers.Adam(), metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(
    imgs, labels,
    epochs=100,  
    batch_size=32
)

Epoch 1/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 79ms/step - accuracy: 0.4236 - loss: 0.8323
Epoch 2/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.5104 - loss: 1.4244
Epoch 3/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - accuracy: 0.5729 - loss: 0.6724
Epoch 4/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 0.5312 - loss: 0.5816
Epoch 5/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 0.5660 - loss: 0.5491
Epoch 6/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.9514 - loss: 0.3247
Epoch 7/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.9340 - loss: 0.3108
Epoch 8/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.8646 - loss: 0.3089
Epoch 9/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 0.9618 - loss: 0.2248
Epoch 10/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 1.0000 - loss: 0.1667
Epoch 11/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 1.0000 - loss: 0.1430
Epoch 12/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 1.0000 - lo

In [6]:
# Rutas para los datos de prueba
test_paths = ["data/test/normal", "data/test/covid19"]
test_imgs = []
test_labels = []

for idx, path in enumerate(test_paths):
    path_images = Path(path).glob("**/*.jpg")
    path_images = [str(x) for x in path_images]
    
    for path_image in path_images:
        img = cv2.imread(path_image, 0)  # Leer en escala de grises
        img = img / 255.0  # Normalizar
        test_imgs.append(cv2.resize(img, (150, 150), cv2.INTER_AREA))  # Redimensionar
    
    test_labels.extend([idx] * len(path_images))  # Añadir etiquetas

# Convertir las listas a arreglos de NumPy
test_imgs = np.array(test_imgs).reshape(-1, 150, 150, 1)
test_labels = np.array(test_labels)

print(f"Test images shape: {test_imgs.shape}")
print(f"Num. of test labels: {len(test_labels)}")

Test images shape: (12, 150, 150, 1)
Num. of test labels: 12


In [7]:
# Evaluar el modelo
loss, accuracy = model.evaluate(test_imgs, test_labels, verbose=1)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step - accuracy: 1.0000 - loss: 0.0405
Test Loss: 0.04047694057226181
Test Accuracy: 1.0


In [ ]:
#from keras.models import load_model
model.save('modelo.keras')

# El siguiente es el que se convertira con tensorflowjs
# model.save('modelo.h5')

In [ ]:
#Cargar el modelo ya entrenado con extensión .keras
#modelo_cargado = load_model('modelo.keras')

In [ ]:
# Función para cargar y preprocesar una imagen
def load_and_preprocess_image(image_path):
    # Cargar la imagen
    img = cv2.imread(image_path, 0)  # 0 para leer en escala de grises

    # Convertir la imagen a escala de grises si tiene 3 canales (RGB)
    if len(img.shape) == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Redimensionar la imagen a (150, 150)
    img = cv2.resize(img, (150, 150))
    # Normalizar la imagen
    img = img / 255.0
    # Añadir una dimensión para los canales (la imagen debe tener forma (150, 150, 1))
    img = np.expand_dims(img, axis=-1)
    # Añadir una dimensión para que la forma sea (1, 150, 150, 1)
    img = np.expand_dims(img, axis=0)
    return img

# Función para hacer predicciones
def predict_image(model, image_path):
    # Cargar y preprocesar la imagen
    img = load_and_preprocess_image(image_path)
    # Realizar la predicción
    predictions = model.predict(img)
    # Obtener la clase con mayor probabilidad
    predicted_class = np.argmax(predictions)

    # Obtener las probabilidades de cada clase
    probabilities = predictions[0]  # predictions es un array de forma (1, 2)

    # Imprimir el resultado con probabilidades
    print(f"Probabilidad de normal: {probabilities[0]:.4f}")
    print(f"Probabilidad de COVID-19: {probabilities[1]:.4f}")

    # Imprimir el resultado
    if predicted_class == 0:
        print("La imagen es de una persona normal (sin COVID-19).")
    else:
        print("La imagen es de una persona con COVID-19.")

In [9]:
# Ejemplo de uso:
image_path = 'data/test/covid19/C25.jpg'
predict_image(model, image_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 184ms/step
Probabilidad de normal: 0.0644
Probabilidad de COVID-19: 0.9356
La imagen es de una persona con COVID-19.
